# NHANES Analysis of Fishbone Variables

In this notebook, we will analyze the correlation among variables and profiles of individuals examined in NHANES survey. The focus is on four variables known as Fishbone: WBC, Hgb, HCT, and PLT (see below).  This work is based on the research of Patrícia Raia Nogueira Cavoto.

### Selecting the driver

The sentence below defines the driver for the database.

In [1]:
%defaultDatasource jdbc:h2:mem:db

# Reference values for NHANES for the 2005-2006 survey

* Extracted from data of the NHANES Web site (https://wwwn.cdc.gov/nchs/nhanes/).

## Importing normal ranges of values indicated in the NHANES documentation

The following query imports the file `reference-ranges.csv` that contains reference ranges from NHANES.

* For each variable it is indicated
  - applicable gender
  - age range (ageStart until ageEnd)

* The range is indicated in the form of mininum and maximum values considered normal.

In [2]:
DROP TABLE IF EXISTS Patients;
CREATE TABLE Patients (
  Id VARCHAR (100) ,
  BIRTHDATE VARCHAR (100) ,
  DEATHDATE VARCHAR(50),
  SSN VARCHAR(50),
  DRIVERS VARCHAR(50),
  PASSPORT VARCHAR(50),
  PREFIX VARCHAR (100) ,
  FIRST VARCHAR (100) ,
  LAST VARCHAR(50),
  SUFFIX VARCHAR(50),
  MAIDEN VARCHAR(50),
  MARITAL VARCHAR(50),
  RACE VARCHAR (100) ,
  ETHNICITY VARCHAR (100) ,
  GENDER VARCHAR(50),
  BIRTHPLACE VARCHAR(100),
  ADDRESS VARCHAR(50),
  CITY VARCHAR(50),
  STATE VARCHAR(50),
  ZIP VARCHAR(50)
) AS SELECT
  Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX, FIRST, LAST,
  SUFFIX ,MAIDEN ,MARITAL ,RACE , ETHNICITY,GENDER ,BIRTHPLACE , ADDRESS,CITY ,STATE ,ZIP
FROM CSVREAD('../data/Simulation/patients.csv');

select * from Patients

In [3]:
DROP TABLE IF EXISTS Medications;
CREATE TABLE Medications (
  START VARCHAR (100) ,
  STOP VARCHAR (100) ,
  PATIENT VARCHAR(50),
  ENCOUNTER VARCHAR(50),
  CODE VARCHAR(50),
  DESCRIPTION VARCHAR(150),
  COST DECIMAL(8,2),
  DISPENSES int(10) ,
  TOTALCOST DECIMAL(9,2),
  REASONCODE VARCHAR(100),
  REASONDESCRIPTION VARCHAR(100)
) AS SELECT
  START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,COST, DISPENSES, TOTALCOST,
  REASONCODE ,REASONDESCRIPTION
FROM CSVREAD('../data/Simulation/medications.csv');

select * from Medications

In [4]:
DROP TABLE IF EXISTS Allergies;
CREATE TABLE Allergies (
  START VARCHAR (100) ,
  STOP VARCHAR (100) ,
  PATIENT VARCHAR(50),
  ENCOUNTER VARCHAR(50),
  CODE VARCHAR(50),
  DESCRIPTION VARCHAR(150)
) AS SELECT
  START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
FROM CSVREAD('../data/Simulation/allergies.csv');

select * from allergies

In [5]:
DROP VIEW Patients_Medications IF EXISTS;
CREATE VIEW Patients_Medications AS

SELECT p.Id pessoa , m.DESCRIPTION medicamento, m.ENCOUNTER ENCOUNTER
FROM Medications m INNER JOIN Patients p 
ON m.PATIENT=p.Id;


select * from Patients_Medications 

In [6]:
DROP VIEW Patients_allergies IF EXISTS;
CREATE VIEW Patients_allergies AS

SELECT p.Id pessoa , m.DESCRIPTION allergie, m.ENCOUNTER ENCOUNTER
FROM allergies m INNER JOIN Patients p 
ON m.PATIENT=p.Id;


select * from Patients_allergies 

In [7]:
DROP VIEW Patients_allergies_Medications IF EXISTS;
CREATE VIEW Patients_allergies_Medications AS

SELECT a.allergie allergie, m.medicamento medicamento
FROM Patients_allergies a INNER JOIN Patients_Medications m 
ON a.pessoa=m.pessoa and a.ENCOUNTER = m.ENCOUNTER; 

select * from Patients_allergies_Medications 

In [8]:
SELECT AVG(NumeroMedicamentos) MediaMedicamentos
FROM (SELECT COUNT(*) Numeromedicamentos
      FROM Patients_Medications
      GROUP BY Pessoa);

9

In [9]:
DROP VIEW Patients_medicamentos_mayor_a_media IF EXISTS;
CREATE VIEW Patients_medicamentos_mayor_a_media AS

SELECT Pessoa, COUNT(*) Numeromedicamentos
FROM Patients_Medications
GROUP BY Pessoa
HAVING Numeromedicamentos > (SELECT AVG(NumeroMedicamentos) MediaMedicamentos
                             FROM (SELECT COUNT(*) Numeromedicamentos
                                   FROM Patients_Medications
                                   GROUP BY Pessoa))

In [10]:
select * from Patients_medicamentos_mayor_a_media;

In [11]:
SELECT AVG(NumeroMedicamentos) MediaMedicamentos
FROM (SELECT COUNT(*) Numeromedicamentos
      FROM Patients_allergies
      GROUP BY Pessoa);

4

In [12]:
DROP VIEW Patients_allergies_mayor_a_media IF EXISTS;
CREATE VIEW Patients_allergies_mayor_a_media AS

SELECT Pessoa, COUNT(*) Numeromedicamentos
FROM Patients_allergies
GROUP BY Pessoa
HAVING Numeromedicamentos > (SELECT AVG(NumeroMedicamentos) MediaMedicamentos
                             FROM (SELECT COUNT(*) Numeromedicamentos
                                   FROM Patients_allergies
                                   GROUP BY Pessoa))

In [13]:
select * from Patients_allergies_mayor_a_media;

In [14]:
DROP VIEW pessoas_mayor_a_la_media IF EXISTS;
CREATE VIEW pessoas_mayor_a_la_media AS
SELECT a.Pessoa
FROM Patients_medicamentos_mayor_a_media a INNER JOIN Patients_allergies_mayor_a_media b 
ON a.Pessoa=b.Pessoa;

In [19]:
DROP VIEW pessoas_allergias_mayor_a_la_media IF EXISTS;
CREATE VIEW pessoas_allergias_mayor_a_la_media AS

SELECT a.pessoa pessoa, m.DESCRIPTION alergia
FROM pessoas_mayor_a_la_media a INNER JOIN allergies m 
ON a.pessoa=m.PATIENT

In [16]:
DROP VIEW pessoas_medicamentos_mayor_a_la_media IF EXISTS;
CREATE VIEW pessoas_medicamentos_mayor_a_la_media AS

SELECT a.pessoa pessoa, m.medicamento medicamento
FROM pessoas_mayor_a_la_media a INNER JOIN Patients_Medications m 
ON a.pessoa=m.pessoa

In [21]:
DROP VIEW UsoMedicamentoB IF EXISTS;
DROP VIEW MedicamentosConjunto IF EXISTS;

CREATE VIEW UsoMedicamentoB AS
SELECT pessoa pessoaB, alergia medicamentoB
FROM pessoas_allergias_mayor_a_la_media;

CREATE VIEW MedicamentosConjunto AS
SELECT U1.alergia medicamentoA, U2.medicamentoB medicamentoB
FROM pessoas_allergias_mayor_a_la_media U1, UsoMedicamentoB U2
WHERE U1.pessoa = U2.pessoaB AND U1.alergia > U2.medicamentoB;

SELECT * FROM MedicamentosConjunto;

In [22]:
SELECT medicamentoA, medicamentoB, COUNT(*) QuantidadeConjunta
FROM MedicamentosConjunto
GROUP BY medicamentoA, medicamentoB;